In [49]:
import pandas as pd
import datetime as dt
import math

pd.set_option('display.max_columns', None) #parametro para apresentar colunas ocultas do dataframe

In [50]:
df1 = pd.read_csv('dfs/1_azul.csv')
df2 = pd.read_csv('dfs/2_amarela.csv')
df3 = pd.read_csv('dfs/3_rosa.csv')
df4 = pd.read_csv('dfs/4_verde.csv')
df5 = pd.read_csv('dfs/5_vermelha.csv')
df6 = pd.read_csv('dfs/6_laranja.csv')

Converter as horas dos Dataframes que estão em formato String para Time

In [51]:
# alterando tipo de dados para data no dataframe 1
for column in df1.columns[:-1]:
    # Convert the Time column to datatime format
    df1[column] = pd.to_datetime(df1[column], format='%H:%M').dt.time
    '''.dt é um acessador (accessor) no pandas que permite acessar
    várias propriedades de um objeto datetime. O .time é uma dessas
    propriedades, que retorna o componente de tempo do objeto datetime.'''
    
# alterando tipo de dados para data no dataframe 2
for column in df2.columns[:-1]:
    # Convert the Time column to datatime format
    df2[column] = pd.to_datetime(df2[column], format='%H:%M').dt.time

# alterando tipo de dados para data no dataframe 3
for column in df3.columns[:-1]:
    # Convert the Time column to datatime format
    df3[column] = pd.to_datetime(df3[column], format='%H:%M').dt.time

# alterando tipo de dados para data no dataframe 4
for column in df4.columns[:-1]:
    # Convert the Time column to datatime format
    df4[column] = pd.to_datetime(df4[column], format='%H:%M').dt.time

# alterando tipo de dados para data no dataframe 5
for column in df5.columns[:-2]:
    # Convert the Time column to datatime format
    df5[column] = pd.to_datetime(df5[column], format='%H:%M').dt.time

# alterando tipo de dados para data no dataframe 6
for column in df6.columns[:-2]:
    # Convert the Time column to datatime format
    df6[column] = pd.to_datetime(df6[column], format='%H:%M').dt.time
    

Forma de calcular a diferença entre datas.

foi adicionado a data atual a hora, pq não é possível, segundo pesquisas, calcular a diferença somente entre horas... 
sendo necessário combinar alguma data, no caso, a data atual (datetime.today()).

Para encontrar a menor diferença de tempo entre uma coluna de horas em um DataFrame pandas e a hora atual, você pode seguir os seguintes passos:

In [52]:

# CLASSE HoraAutocarro()

class HoraAutocarro:
    def __init__(self, nome_paragem, nome_rota):
        self.nome_paragem = nome_paragem
        self.nome_rota = nome_rota
        
    def ProximoAutocarro(self):
        #verificar se existe a paragem para determinada linha
        if self.nome_paragem not in self.nome_rota.columns:
            return 'Não constam dados nesta linha sobre esta paragem'
        
        #Parametros de tempo do metodo:
        hora_atual = dt.datetime.now()
        #ultima_hora = self.nome_rota[self.nome_paragem].tolist()
        # Utilizando list comprehension para filtrar os valores "NaN"
        #ultima_hora =  list(filter(lambda x: isinstance(x, dt.time), ultima_hora))

        #Dataframes filtrados:
        #Filtragem dos valores NAN, caso existam.
        if self.nome_rota[self.nome_paragem].isna().any():
            df_filtrado1 = self.nome_rota[~self.nome_rota[self.nome_paragem].isna()]
            df_filtrado = df_filtrado1[df_filtrado1[self.nome_paragem] >= hora_atual.time()]
        else:
            df_filtrado = self.nome_rota[self.nome_rota[self.nome_paragem] >= hora_atual.time()]

        # Filtrando a última hora da série com base no dia da semana.
        if hora_atual.weekday() < 5: #hj é dia da semana!
            df_week = df_filtrado.query('sabado  == 0') 
            ultima_hora = df_week[self.nome_paragem].tolist()
            # Utilizando list comprehension para filtrar os valores "NaN"
            ultima_hora =  list(filter(lambda x: isinstance(x, dt.time), ultima_hora))

        else:
            df_weekend = df_filtrado.query('sabado  == 1') #hj é sábado!
            ultima_hora = df_weekend[self.nome_paragem].tolist()
            # Utilizando list comprehension para filtrar os valores "NaN"
            ultima_hora =  list(filter(lambda x: isinstance(x, dt.time), ultima_hora))

        #Filtragem (retirada do df) das horas inferiores a hora atual.
        #df_filtrado = self.nome_rota[self.nome_rota[self.nome_paragem] >= hora_atual.time()]
        #df_filtrado = df_filtrado[df_filtrado[self.nome_paragem] >= hora_atual.time()]

        
        #Verificação de saidas para as variações de tempo e dados de entrada

        #Verificar o dia da semana para filtrar a base de dados
        if hora_atual.weekday() == 6: #hj é domingo!
            return "Não existem autocarros para o dia de hoje!"

        elif hora_atual.weekday() == 5: #hj é sábado!
            df_sabado = self.nome_rota[~self.nome_rota[self.nome_paragem].isna()].query('sabado  == 1')#df_filtrado.query('sabado  == 1') -> ALTERAÇÃO NA LINHA DE CODIGO
            if df_sabado.shape[0] == 0:
                return "Segundo a tabela de horários, aos sábados não constam autocarros para esta paragem."
            
            elif hora_atual.time() > ultima_hora[-1]: #verificar se o último autocarro já passou pela paragem: -> MUDANÇA DE POSIÇÃO DO ELIF     (bug no cod quando o horario passa da ultima hora dos autocarros)
                return "Segundo a tabela de horários, o último autocarro já passou por esta paragem."
            
            else:
                # Df com os valores para sábado, excluindo as linhas com as horas inferios a hora atual
                df_sabado_filtrado = df_filtrado[~df_filtrado[self.nome_paragem].isna()].query('sabado  == 1')

                # Calcular a diferença de tempo entre cada valor na coluna de horários da paragem e a hora atual
                df_sabado_filtrado['diferenca'] = df_sabado_filtrado[self.nome_paragem].apply(lambda x: dt.datetime.combine(dt.datetime.today(), x) - dt.datetime.combine(dt.datetime.today(), hora_atual.time()))

                # Encontrar a menor diferença de tempo
                menor_diferenca = df_sabado_filtrado['diferenca'].min()

                # Encontrar o índice da hora mais próxima
                indice_hora_proxima = df_sabado_filtrado['diferenca'].idxmin()

                # Obter a hora mais próxima
                hora_proxima = df_sabado_filtrado.loc[indice_hora_proxima, self.nome_paragem]

                #Retorno do proximo autocarro"
                return f"{hora_proxima.hour:02d}:{hora_proxima.minute:02d}"
            
        #elif hora_atual.time() > ultima_hora[-1]: #verificar se o último autocarro já passou pela paragem
           # return "Segundo a tabela de horários,  o último autocarro já passou por esta paragem."
        
        elif hora_atual.weekday() < 5: #hj é dia da semana!
            df_semana = df_filtrado.query('sabado  == 0')
        
            # Calcular a diferença de tempo entre cada valor na coluna de horários da paragem e a hora atual
            df_semana['diferenca'] = df_semana[self.nome_paragem].apply(lambda x: dt.datetime.combine(dt.datetime.today(), x) - dt.datetime.combine(dt.datetime.today(), hora_atual.time()))

            # Encontrar o índice da hora mais próxima
            indice_hora_proxima = df_semana['diferenca'].idxmin()

            # Obter a hora mais próxima
            hora_proxima = df_semana.loc[indice_hora_proxima, self.nome_paragem]


            #Retorno do proximo autocarro"
            return f"{hora_proxima.hour:02d}:{hora_proxima.minute:02d}"
            


    def TempoEspera(self):
        #verificar se existe a paragem para determinada linha
        if self.nome_paragem not in self.nome_rota.columns:
            return ""
            #return 'Não constam dados nesta linha sobre esta paragem'
        
        #Parametros de tempo do metodo:
        #agora = datetime.datetime.now() #### TESTAR DEPOIS DE A VARIAVEL AGORA VIRAR SOMENTE A HORA VARIAVEL HORA_ATUAL
        hora_atual = dt.datetime.now()
        ultima_hora = self.nome_rota[self.nome_paragem].tolist()
        # Utilizando list comprehension para filtrar os valores "NaN"
        ultima_hora =  list(filter(lambda x: isinstance(x, dt.time), ultima_hora))

        #Dataframes filtrados:
        #Filtragem dos valores NAN, caso existam e filtragem (retirada do df) das horas inferiores a hora atual.

        if self.nome_rota[self.nome_paragem].isna().any():
            df_filtrado = self.nome_rota[~self.nome_rota[self.nome_paragem].isna()]
            df_filtrado = df_filtrado[df_filtrado[self.nome_paragem] >= hora_atual.time()]
            
        else:
            df_filtrado = self.nome_rota[self.nome_rota[self.nome_paragem] >= hora_atual.time()]        
        #df_filtrado = self.nome_rota[self.nome_rota[self.nome_paragem] >= hora_atual.time()]

        #Verificação de saidas para as variações de tempo e dados de entrada

        #Verificar o dia da semana para filtrar a base de dados
        if hora_atual.weekday() == 6: #hj é domingo!
            return ""
            #return "Não existem autocarros para o dia de hoje!"

        elif hora_atual.weekday() == 5: #hj é sábado!
            df_sabado = self.nome_rota[~self.nome_rota[self.nome_paragem].isna()].query('sabado  == 1') #df_sabado = df_filtrado.query('sabado  == 1')
            if df_sabado.shape[0] == 0:
                return ""
                #return "Segundo a tabela de horários, aos sábados não constam autocarros para esta paragem."
                
            elif hora_atual.time() > ultima_hora[-1]: #verificar se o último autocarro já passou pela paragem
                return "g"
                #return "Segundo a tabela de horários,  o último autocarro já passou por esta paragem."

            else:
                # Calcular a diferença de tempo entre cada valor na coluna de horários da paragem e a hora atual
                df_sabado['diferenca'] = df_sabado[self.nome_paragem].apply(lambda x: dt.datetime.combine(dt.datetime.today(), x) - dt.datetime.combine(dt.datetime.today(), hora_atual.time()))

                # Encontrar a menor diferença de tempo
                menor_diferenca = df_sabado['diferenca'].min()
            
                hora= int(menor_diferenca.total_seconds() // 3600)
                minuto = int((menor_diferenca.total_seconds() // 60) % 60)

                #Retorno do tempo de espera"
                return f"{hora:02d}:{minuto:02d}min."
            
        
        elif hora_atual.weekday() < 5: #hj é dia da semana!
            df_semana = df_filtrado.query('sabado  == 0')            
            
        
            # Calcular a diferença de tempo entre cada valor na coluna de horários da paragem e a hora atual
            df_semana['diferenca'] = df_semana[self.nome_paragem].apply(lambda x: dt.datetime.combine(dt.datetime.today(), x) - dt.datetime.combine(dt.datetime.today(), hora_atual.time()))

            # Encontrar a menor diferença de tempo
            menor_diferenca = df_semana['diferenca'].min()
          
            hora= int(menor_diferenca.total_seconds() // 3600)
            minuto = int((menor_diferenca.total_seconds() // 60) % 60)

            #Retorno do tempo de espera"
            return f"{hora:02d}:{minuto:02d}min."
            






In [53]:
autocarro = 'CENTRAL DE CAMIONAGEM (IDA)' #'AV. DR. EGAS MONIZ (TAMBOREIRA).1'
df = df3

teste = HoraAutocarro(autocarro,df)

print(teste.ProximoAutocarro())

print(teste.TempoEspera())

09:30
-1:19min.


In [54]:
hora_atual = dt.datetime.now()

if df[autocarro].isna().any():
    df_filtrado1 = df[~df[autocarro].isna()]
    df_filtrado = df_filtrado1[df_filtrado1[autocarro] >= hora_atual.time()]
else:
    df_filtrado = df[df[autocarro] >= hora_atual.time()]

df_sabado_filtrado = df_filtrado[~df_filtrado[autocarro].isna()].query('sabado  == 1')
df_sabado_filtrado.head(50)

#df_filtrado.head(50)

,CENTRAL DE CAMIONAGEM (IDA),LARGO DE CAMÕES (IDA),AV. DR. ALFR. SOUSA (MULTIUSOS),AV. DR. ALFREDO SOUSA (CTT),MEIA LARANJA,LARGO DR. JOÃO DE ALMEIDA,ESCOLA SECUNDÁRIA DA SÉ,AV. DR. EGAS MONIZ (TAMBOREIRA),LECLERC (IDA),ESCOLA RURAL,HOSPITAL,CALVILHE,CEPÕES (CRUZ.),BRITIANDE C. BARRO,BRITIANDE (FARMÁCIA),ZONA INDUSTRIAL,VÁRZEA DE ABRUNHAIS (EIRÓ),SANTO ALEIXO,FUNDO DE VILA (ESCOLA),ALVÉLOS,HOSPITAL.1,ESCOLA RURAL.1,LECLERC (VOLTA),BALSEMÃO (CRUZ.),LARGO DO CHAFARIZ,RUA DA CALÇADA,AV. DR. EGAS MONIZ (TAMBOREIRA).1,ESC. SEC. DA SÉ/R. DA CALÇADA,CASA DA LAVOURA,CASA DAS BROLHAS,LARGO DE CAMÕES (VOLTA),CENTRAL DE CAMIONAGEM (VOLTA),sabado
9,09:30:00,09:31:00,09:32:00,09:33:00,09:34:00,09:35:00,09:36:00,09:37:00,09:38:00,09:39:00,09:40:00,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,09:40:00,09:41:00,09:42:00,09:43:00,09:44:00,09:46:00,NaT,NaT,09:47:00,09:48:00,09:49:00,09:50:00,1
10,14:30:00,14:31:00,14:32:00,14:33:00,14:34:00,14:35:00,14:36:00,14:37:00,14:38:00,14:39:00,14:40:00,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,14:40:00,14:41:00,14:42:00,14:43:00,14:44:00,14:46:00,NaT,NaT,14:47:00,14:48:00,14:49:00,14:50:00,1
11,15:30:00,15:31:00,15:32:00,15:33:00,15:34:00,15:35:00,15:36:00,15:37:00,15:38:00,15:39:00,15:40:00,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,15:40:00,15:41:00,15:42:00,15:43:00,15:44:00,15:46:00,NaT,NaT,15:47:00,15:48:00,15:49:00,15:50:00,1


In [55]:
if hora_atual.weekday() < 5: #hj é dia da semana!
    df_week = df_filtrado.query('sabado  == 0') 
    ultima_hora = df_week[autocarro].tolist()
    # Utilizando list comprehension para filtrar os valores "NaN"
    ultima_hora =  list(filter(lambda x: isinstance(x, dt.time), ultima_hora))

else: 
    df_weekend = df_filtrado.query('sabado  == 1')   
    ultima_hora = df_weekend[autocarro].tolist()
    # Utilizando list comprehension para filtrar os valores "NaN"
    ultima_hora =  list(filter(lambda x: isinstance(x, dt.time), ultima_hora))


print(ultima_hora[-1])



15:30:00


In [56]:
df.head(100)

,CENTRAL DE CAMIONAGEM (IDA),LARGO DE CAMÕES (IDA),AV. DR. ALFR. SOUSA (MULTIUSOS),AV. DR. ALFREDO SOUSA (CTT),MEIA LARANJA,LARGO DR. JOÃO DE ALMEIDA,ESCOLA SECUNDÁRIA DA SÉ,AV. DR. EGAS MONIZ (TAMBOREIRA),LECLERC (IDA),ESCOLA RURAL,HOSPITAL,CALVILHE,CEPÕES (CRUZ.),BRITIANDE C. BARRO,BRITIANDE (FARMÁCIA),ZONA INDUSTRIAL,VÁRZEA DE ABRUNHAIS (EIRÓ),SANTO ALEIXO,FUNDO DE VILA (ESCOLA),ALVÉLOS,HOSPITAL.1,ESCOLA RURAL.1,LECLERC (VOLTA),BALSEMÃO (CRUZ.),LARGO DO CHAFARIZ,RUA DA CALÇADA,AV. DR. EGAS MONIZ (TAMBOREIRA).1,ESC. SEC. DA SÉ/R. DA CALÇADA,CASA DA LAVOURA,CASA DAS BROLHAS,LARGO DE CAMÕES (VOLTA),CENTRAL DE CAMIONAGEM (VOLTA),sabado
0,07:30:00,07:31:00,07:32:00,07:33:00,07:34:00,07:35:00,07:36:00,07:37:00,07:38:00,07:39:00,07:40:00,NaT,NaT,07:42:00,07:43:00,07:45:00,07:49:00,07:52:00,07:55:00,07:58:00,08:03:00,08:04:00,08:05:00,NaT,NaT,NaT,08:06:00,08:07:00,08:08:00,08:09:00,08:10:00,08:11:00,0
1,08:30:00,08:31:00,08:32:00,08:33:00,08:34:00,08:35:00,08:36:00,08:37:00,08:38:00,08:39:00,08:40:00,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,08:40:00,08:41:00,08:42:00,08:43:00,08:44:00,08:46:00,NaT,NaT,08:47:00,08:48:00,08:49:00,08:50:00,0
2,09:30:00,09:31:00,09:32:00,09:33:00,09:34:00,09:35:00,09:36:00,09:37:00,09:38:00,09:39:00,09:40:00,09:41:00,09:42:00,09:44:00,09:45:00,NaT,NaT,NaT,NaT,NaT,09:49:00,09:50:00,09:51:00,09:52:00,09:53:00,09:55:00,NaT,NaT,09:56:00,09:57:00,09:58:00,09:59:00,0
3,12:15:00,12:16:00,12:17:00,12:18:00,12:19:00,12:20:00,12:21:00,12:22:00,12:23:00,12:24:00,12:25:00,12:26:00,12:27:00,12:29:00,12:30:00,12:32:00,12:36:00,12:39:00,12:42:00,12:45:00,12:50:00,12:51:00,12:52:00,NaT,NaT,NaT,12:53:00,12:54:00,12:55:00,12:56:00,12:57:00,12:58:00,0
4,13:30:00,13:31:00,13:32:00,13:33:00,13:34:00,13:35:00,13:36:00,13:37:00,13:38:00,13:39:00,13:40:00,13:41:00,13:42:00,13:44:00,13:45:00,13:47:00,13:51:00,13:54:00,13:57:00,14:00:00,14:05:00,14:06:00,14:07:00,NaT,NaT,NaT,14:08:00,14:09:00,14:10:00,14:11:00,14:12:00,14:13:00,0
5,16:30:00,16:31:00,16:32:00,16:33:00,16:34:00,16:35:00,16:36:00,16:37:00,16:38:00,16:39:00,16:40:00,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,16:40:00,16:41:00,16:42:00,16:43:00,16:44:00,16:46:00,NaT,NaT,16:47:00,16:48:00,16:49:00,16:50:00,0
6,17:30:00,17:31:00,17:32:00,17:33:00,17:34:00,17:35:00,17:36:00,17:37:00,17:38:00,17:39:00,17:40:00,17:41:00,17:42:00,17:44:00,17:45:00,17:47:00,17:51:00,17:54:00,17:57:00,18:00:00,18:05:00,18:06:00,18:07:00,NaT,NaT,NaT,18:08:00,18:09:00,18:10:00,18:11:00,18:12:00,18:13:00,0
7,18:30:00,18:31:00,18:32:00,18:33:00,18:34:00,18:35:00,18:36:00,18:37:00,18:38:00,18:39:00,18:40:00,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,18:40:00,18:41:00,18:42:00,18:43:00,18:44:00,18:46:00,NaT,NaT,18:47:00,18:48:00,18:49:00,18:50:00,0
8,07:30:00,07:31:00,07:32:00,07:33:00,07:34:00,07:35:00,07:36:00,07:37:00,07:38:00,07:39:00,07:40:00,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,07:40:00,07:41:00,07:42:00,07:43:00,07:44:00,07:46:00,NaT,NaT,07:47:00,07:48:00,07:49:00,07:50:00,1
9,09:30:00,09:31:00,09:32:00,09:33:00,09:34:00,09:35:00,09:36:00,09:37:00,09:38:00,09:39:00,09:40:00,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,09:40:00,09:41:00,09:42:00,09:43:00,09:44:00,09:46:00,NaT,NaT,09:47:00,09:48:00,09:49:00,09:50:00,1


In [57]:
df5['CENTRAL DE CAMIONAGEM (IDA)']

0    08:15:00
1    09:15:00
2    12:30:00
3    14:15:00
4    18:30:00
5    09:00:00
6    12:30:00
Name: CENTRAL DE CAMIONAGEM (IDA), dtype: object

In [58]:
#Testes para criar uma formatação do POPUP


def FormatTexto(texto):
    if texto == "Não constam dados nesta linha sobre esta paragem":
        pass
    else:
        return texto



# Obter a hora atual
def HoraAutocarro(nome_paragem,nome_rota):

    #verificar se existe a paragem para determinada linha
    if nome_paragem not in nome_rota.columns:
        return 'Não constam dados nesta linha sobre esta paragem'
    

    agora = datetime.now()
    hora_atual = agora
    #hora_atual = agora + timedelta(days=-1) #DIA ADIANTADO
    #hora_atual = agora + timedelta(hours=6) #HORA ADIANTADA
    ultima_hora = nome_rota[nome_paragem].tolist()


    #filtragem (retirada) das horas inferiores a hora atual
    df_filtrado = nome_rota[nome_rota[nome_paragem] >= hora_atual.time()]

    #verificar o dia da semana para filtrar a base de dados
    if hora_atual.weekday() == 6: #hj é domingo!
        print("Não existem autocarros para o dia de hoje!)")

    elif hora_atual.weekday() == 5: #hj é sábado!
        df_sabado = df_filtrado.query('sabado  == 1')
        if df_sabado.shape[0] == 0:
            print('Segundo a tabela de horários, aos sábados não constam autocarros para esta paragem.')

        # Calcular a diferença de tempo entre cada valor na coluna de horários da paragem e a hora atual
        df_sabado['diferenca'] = df_sabado[nome_paragem].apply(lambda x: datetime.combine(datetime.today(), x) - datetime.combine(datetime.today(), hora_atual.time()))

        # Encontrar a menor diferença de tempo
        menor_diferenca = df_sabado['diferenca'].min()

        # Encontrar o índice da hora mais próxima
        indice_hora_proxima = df_sabado['diferenca'].idxmin()

        # Obter a hora mais próxima
        hora_proxima = df_sabado.loc[indice_hora_proxima, nome_paragem]

        # Exibir a menor diferença de tempo e a hora mais próxima
        print("Hora atual: ", hora_atual.time())
        print("Tempo de espera:", menor_diferenca)
        print("Próximo autocarro:", hora_proxima)

    elif hora_atual.time() > ultima_hora[-1]: #verificar se o último autocarro já passou pela paragem
        print('Segundo a tabela de horários,  o último autocarro já passou por esta paragem.')
        
    elif hora_atual.weekday() < 5: #hj é dia da semana!
        df_semana = df_filtrado.query('sabado  == 0')
        
        # Calcular a diferença de tempo entre cada valor na coluna de horários da paragem e a hora atual
        df_semana['diferenca'] = df_semana[nome_paragem].apply(lambda x: datetime.combine(datetime.today(), x) - datetime.combine(datetime.today(), hora_atual.time()))

        # Encontrar a menor diferença de tempo
        menor_diferenca = df_semana['diferenca'].min()

        # Encontrar o índice da hora mais próxima
        indice_hora_proxima = df_semana['diferenca'].idxmin()

        # Obter a hora mais próxima
        hora_proxima = df_semana.loc[indice_hora_proxima, nome_paragem]

        # Exibir a menor diferença de tempo e a hora mais próxima
        #print("dia da semana: ", hora_atual.weekday())
        #print("Hora atual: ", hora_atual.time())
        #print("Tempo de espera:", menor_diferenca)
        #print("Próximo autocarro:", hora_proxima)

        hora= menor_diferenca.seconds // 3600
        minuto = (menor_diferenca.seconds // 60) % 60

        #return "Próximo autocarro: "
        return "Tempo de espera: "+ f"{hora:02d}:{minuto:02d}min."
       



In [59]:
'''
lista1 = df1.columns[:-1].tolist()
lista2 = df2.columns[:-1].tolist()
lista3 = df3.columns[:-1].tolist()
lista4 = df4.columns[:-1].tolist()
lista5 = df5.columns[:-2].tolist()
lista6 = df6.columns[:-2].tolist()

lista_total= lista1 + lista2 + lista3 + lista4 + lista5 + lista6

lista_total = list(set(lista_total)) #deletando valores duplicados


minha_string = nome_paragem
presente = False


for nome_paragem in lista_total:
    if minha_string == nome_paragem:
        presente = True
        break

if presente == False:
    print("Paragem não existe.")
'''

'\nlista1 = df1.columns[:-1].tolist()\nlista2 = df2.columns[:-1].tolist()\nlista3 = df3.columns[:-1].tolist()\nlista4 = df4.columns[:-1].tolist()\nlista5 = df5.columns[:-2].tolist()\nlista6 = df6.columns[:-2].tolist()\n\nlista_total= lista1 + lista2 + lista3 + lista4 + lista5 + lista6\n\nlista_total = list(set(lista_total)) #deletando valores duplicados\n\n\nminha_string = nome_paragem\npresente = False\n\n\nfor nome_paragem in lista_total:\n    if minha_string == nome_paragem:\n        presente = True\n        break\n\nif presente == False:\n    print("Paragem não existe.")\n'

Visualizar os dados em um mapa Folium


In [60]:
import folium
import geopandas as gpd
from shapely import Point

paragem = gpd.read_file("./shp/paragem_autocarro_lamego.shp")
linhas = gpd.read_file("./shp/rota_autocarro.shp")

tabela_horarios = "https://www.transdev.pt/sites/default/files/pdf-sim/verdinho_info_site_1801.pdf"

linha_amarela = linhas.query("linha == 'amarela'")
linha_azul = linhas.query("linha == 'azul'")
linha_rosa = linhas.query("linha == 'rosa'")
linha_verde = linhas.query("linha == 'verde'")
linha_vermelha = linhas.query("linha == 'vermelha'")
linha_laranja = linhas.query("linha == 'laranja'")

m = folium.Map(#width=600, height=600,
           location=[paragem.geometry.y.mean(), paragem.geometry.x.mean()],
           zoom_start=14,
           )

#add novo basemap
#folium.TileLayer('stamenterrain', attr="stamenterrain").add_to(m)
#folium.TileLayer('stamenwatercolor', attr="stamenwatercolor").add_to(m)
'''folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = False,
        control = True
       ).add_to(m)'''

folium.TileLayer(tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
                 attr='Google Satellite',
                 name='Google Satellite',
                 overlay=False).add_to(m)

folium.TileLayer(tiles='https://mt1.google.com/vt/lyrs=s,h&x={x}&y={y}&z={z}',
                 attr='Google Satellite with Labels',
                 name='Google Satellite com rótulos',
                 overlay=False
                 ).add_to(m)

# grupo de layers
shapesparegem = folium.FeatureGroup(name= 'Paragem').add_to(m)

'''folium.GeoJson(
    paragem,
    popup= 'teste',
    zoom_on_click=True, #centraliza a pag no ponto clicado
    style_function=lambda feature: {
        'fillColor': 'green',
        'color': 'darkred',
        'weight': 0.1,
        'icon': 'bus'
    }
).add_to(m)'''

#shape de paragens
for id in range(paragem.shape[0]):
    #hora_autocarro_df1 = HoraAutocarro(paragem['nome'][id],df1)
    autocarro_df1 = HoraAutocarro(paragem['nome'][id],df1)
    autocarro_df2 = HoraAutocarro(paragem['nome'][id],df2)
    autocarro_df3 = HoraAutocarro(paragem['nome'][id],df3)
    autocarro_df4 = HoraAutocarro(paragem['nome'][id],df4)
    autocarro_df5 = HoraAutocarro(paragem['nome'][id],df5)
    autocarro_df6 = HoraAutocarro(paragem['nome'][id],df6)
    

    link_google_maps = 'http://www.google.com/maps/place/'+ str(paragem['lat'][id]) + ',' + str(paragem['long'][id])
    location = [paragem['geometry'][id].y, paragem['geometry'][id].x]
    icon = folium.features.CustomIcon("./img/icon_bus.png", icon_size=(20,20)) 
    popup = folium.Popup(
        "<strong>Tempo de espera:</strong>" +
        "<br><br><strong>L1 - Azul:</strong> " + autocarro_df1.ProximoAutocarro() + " <strong>(" + autocarro_df1.TempoEspera() + ")</strong>" +
        "<br><strong>L2 - Amarela:</strong> " + autocarro_df2.ProximoAutocarro() + " <strong>(" + autocarro_df2.TempoEspera() + ")</strong>" +
        "<br><strong>L3 - Rosa:</strong> " + autocarro_df3.ProximoAutocarro() + " <strong>(" + autocarro_df3.TempoEspera() + ")</strong>" +
        "<br><strong>L4 - Verde:</strong> " + autocarro_df4.ProximoAutocarro() + " <strong>(" + autocarro_df4.TempoEspera() + ")</strong>" +
        "<br><strong>L5 - Vermelha:</strong> " + autocarro_df5.ProximoAutocarro() + " <strong>(" + autocarro_df5.TempoEspera() + ")</strong>" +
        "<br><strong>L6 - Laranja:</strong> " + autocarro_df6.ProximoAutocarro() + " <strong>(" + autocarro_df6.TempoEspera() + ")</strong>" +
        "<br><br>Link Google Maps (localização): <a href='" + link_google_maps + "'>link</a> <br>" + 
        "Link Tabela de Horários Verdinho (Transdev): <a href='" + tabela_horarios + "'>link</a>",
        max_width=580
    )

    marker= folium.Marker(
        location=location,
        popup=popup,
        tooltip='<strong>'+ str(paragem['nome'][id]) + '</strong>',
        icon=icon

        

    ).add_to(shapesparegem)




# Função para definir o estilo com base em um atributo
def style_function(feature):
    color = '#000000'  # Cor padrão (preto)
    
    # Obter o valor do atributo desejado
    attribute_value = feature['properties']['linha']
    
    # Definir a cor com base no valor do atributo
    if attribute_value == 'amarela':
        color = '#F9C900'  # amarela
    elif attribute_value == 'verde':
        color = '#BFD611'  # Verde
    elif attribute_value == 'azul':
        color = '#38A2E6'  # azul
    elif attribute_value == 'rosa':
        color = '#E3008D'  # rosa
    elif attribute_value == 'vermelha':
        color = '#E41619'  # vermelha
    elif attribute_value == 'laranja':
        color = '#EB7806'  # laranja
    # Adicione mais condições if/elif conforme necessário para mapear outros valores do atributo
    
    return {'color': color, 'weight': 2}  # Retornar um dicionário de estilo com a cor e a espessura da linha


# Adicionar as linhas ao mapa com o estilo definido pela função style_function
folium.GeoJson(linha_azul, style_function=style_function, name='Linha 1 -  Azul', show=False,tooltip='Linha 1').add_to(m)
folium.GeoJson(linha_amarela, style_function=style_function, name='Linha 2 -  Amarela', show=False,tooltip='Linha 2').add_to(m)
folium.GeoJson(linha_rosa, style_function=style_function, name='Linha 3 -  Rosa', show=False,tooltip='Linha 3').add_to(m)
folium.GeoJson(linha_verde, style_function=style_function, name='Linha 4 -  Verde', show=False,tooltip='Linha 4').add_to(m)
folium.GeoJson(linha_vermelha, style_function=style_function, name='Linha 5 -  Vermelha', show=False,tooltip='Linha 4').add_to(m)
folium.GeoJson(linha_laranja, style_function=style_function, name='Linha 6 -  Laranja', show=False,tooltip='Linha 5').add_to(m)



folium.LayerControl().add_to(m)

m.save('output.html')


